# Galaxy Cuts

Notebook experiments with looking at cutting the galaxies below the mass resolution from the saved pickeled data.

In [1]:
cd /Users/scottclay/RESEARCH/lgalaxies/Lgalaxies_Analysis/read_data_pandas_hdf5/

/Users/scottclay/RESEARCH/lgalaxies/Lgalaxies_Analysis/read_data_pandas_hdf5


In [1]:
import sys
datadir = '../../Hen15_Dustmodel/output/'
sys.path.insert(0,datadir)

In [2]:
# numpy dtype for LGAL_GAL_STRUCT
import numpy as np
struct_dtype = np.dtype([
('Type',np.int32,1),
('HaloIndex',np.int32,1),
('SnapNum',np.int32,1),
('LookBackTimeToSnap',np.float32,1),
('CentralMvir',np.float32,1),
('CentralRvir',np.float32,1),
('DistanceToCentralGal',np.float32,3),
('Pos',np.float32,3),
('Vel',np.float32,3),
('Len',np.int32,1),
('Mvir',np.float32,1),
('Rvir',np.float32,1),
('Vvir',np.float32,1),
('Vmax',np.float32,1),
('GasSpin',np.float32,3),
('StellarSpin',np.float32,3),
('InfallVmax',np.float32,1),
('InfallVmaxPeak',np.float32,1),
('InfallSnap',np.int32,1),
('InfallHotGas',np.float32,1),
('HotRadius',np.float32,1),
('OriMergTime',np.float32,1),
('MergTime',np.float32,1),
('ColdGas',np.float32,1),
('StellarMass',np.float32,1),
('BulgeMass',np.float32,1),
('DiskMass',np.float32,1),
('HotGas',np.float32,1),
('EjectedMass',np.float32,1),
('BlackHoleMass',np.float32,1),
('ICM',np.float32,1),
('MetalsColdGas',np.float32,3),
('MetalsBulgeMass',np.float32,3),
('MetalsDiskMass',np.float32,3),
('MetalsHotGas',np.float32,3),
('MetalsEjectedMass',np.float32,3),
('MetalsICM',np.float32,3),
('PrimordialAccretionRate',np.float32,1),
('CoolingRadius',np.float32,1),
('CoolingRate',np.float32,1),
('CoolingRate_beforeAGN',np.float32,1),
('QuasarAccretionRate',np.float32,1),
('RadioAccretionRate',np.float32,1),
('Sfr',np.float32,1),
('SfrBulge',np.float32,1),
('XrayLum',np.float32,1),
('BulgeSize',np.float32,1),
('StellarDiskRadius',np.float32,1),
('GasDiskRadius',np.float32,1),
('CosInclination',np.float32,1),
('DisruptOn',np.int32,1),
('MergeOn',np.int32,1),
('MagDust',np.float32,2),
('Mag',np.float32,2),
('MagBulge',np.float32,2),
('MassWeightAge',np.float32,1),
('DiskMass_elements',np.float32,11),
('BulgeMass_elements',np.float32,11),
('ColdGas_elements',np.float32,11),
('HotGas_elements',np.float32,11),
('ICM_elements',np.float32,11),
('EjectedMass_elements',np.float32,11),
('DustRatesISM',np.float32,5),
('Dust_elements',np.float32,11),
('Attenuation_Dust',np.float32,1)
])

properties_used = {}
for ii in struct_dtype.names:
    properties_used[ii] = False



In [3]:
def read_snap(folder,file_prefix,firstfile,lastfile,props,template):
    """ Reads L-Galaxy output files.
    Returns: (nTrees,nHalos,nTreeHalos,gals)
    Inputs: (folder,file_prefix,firstfile,lastfile,props,template)
    props - list of properties to return
    template - structure dtype definition for database """
    filter_list = []
    for prop in props:
        if props[prop]:
            filter_list.append((prop,template[prop]))
    filter_dtype = np.dtype(filter_list)
    # First loop to determine how many galaxies there are:
    nTrees = 0
    nHalos = 0
    for ifile in range(firstfile,lastfile+1):
        filename = folder+'/'+file_prefix+"_"+"%d"%(ifile)
        f = open(filename,"rb")
        this_nTrees =  np.fromfile(f,np.int32,1)[0]
        nTrees += this_nTrees
        this_nHalos = np.fromfile(f,np.int32,1)[0]
        nHalos += this_nHalos
        f.close()
    # Allocate arrays
    print("Total nGals = ",nHalos)
    nTreeHalos = np.empty(nTrees,dtype=np.int32)
    gals = np.empty(nHalos,dtype=filter_dtype)
    # Second loop to populate arrays
    nTrees = 0
    nHalos = 0
    for ifile in range(firstfile,lastfile+1):
        filename = folder+'/'+file_prefix+"_"+"%d"%(ifile)
        f = open(filename,"rb")
        this_nTrees =  np.fromfile(f,np.int32,1)[0]
        this_nHalos = np.fromfile(f,np.int32,1)[0]
        print("File ", ifile," nGals = ",this_nHalos)
        addednTreeHalos = np.fromfile(f,np.int32,this_nTrees)
        nTreeHalos[nTrees:nTrees+this_nTrees]=addednTreeHalos
        this_addedGalaxy = np.fromfile(f,template,this_nHalos) # all properties
        addedGalaxy = np.empty(this_nHalos,dtype=filter_dtype) # selected props
        for prop in template.names:
            if props[prop]:
#                try:
                addedGalaxy[prop] = this_addedGalaxy[prop]
#                except:
#                    embed()
        gals[nHalos:nHalos+this_nHalos] = addedGalaxy
        nTrees += this_nTrees
        nHalos += this_nHalos
        f.close()
    return (nTrees,nHalos,nTreeHalos,gals)

In [4]:
snaplist_file = '../MRPlancksnaplist.txt'

In [5]:
snapshot=15
file_prefix = "SA_z6.97"
output_file = "lgal_z7.pkl"

snapshot=58
file_prefix = "SA_z0.00"
output_file = "lgal_z0.pkl"



In [8]:

# Define what properties you want to read in
props = properties_used

props['Type'] = True
props['ColdGas'] = True
props['StellarMass'] = True
props['BulgeMass'] = True
props['DiskMass'] = True
props['HotGas'] = True
props['ICM'] = True
props['MetalsColdGas'] = True
props['MetalsBulgeMass'] = True
props['MetalsDiskMass'] = True
props['MetalsHotGas'] = True
props['MetalsEjectedMass'] = True
props['MetalsICM'] = True
props['Sfr'] = True
props['SfrBulge'] = True
props['DiskMass_elements'] = True
props['BulgeMass_elements'] = True
props['ColdGas_elements'] = True
props['HotGas_elements'] = True
#props['DustMassISM'] = True
props['DustRatesISM'] = True
props['Dust_elements'] = True
props['Attenuation_Dust'] = True
props['Mag'] = True
props['MagDust'] = True
props['GasDiskRadius'] = True

# MRII

In [9]:
# Define which files you want to read in
firstfile = 40
lastfile = 40 #511

(nTrees,nHalos,nTreeHalos,gals) = \
    read_snap(datadir,file_prefix,firstfile,lastfile,\
                            props,struct_dtype)

Total nGals =  24614
File  40  nGals =  24614


In [10]:
print('Number of galaxies in MRII:',len(gals))


Number of galaxies in MRII: 24614


In [11]:
len(gals[gals['StellarMass']>0.0]['StellarMass']*1.0E10/0.673)

9570

In [12]:
len(gals[gals['StellarMass']*1.0E10/0.673>1.0E7]['StellarMass']*1.0E10/0.673)

1379

# MR

In [13]:
# Define which files you want to read in
firstfile = 5
lastfile = 5 #511

(nTrees,nHalos,nTreeHalos,gals) = \
    read_snap(datadir,file_prefix,firstfile,lastfile,\
                            props,struct_dtype)

Total nGals =  45631
File  5  nGals =  45631


In [14]:
print('Number of galaxies in MR:',len(gals))

Number of galaxies in MR: 45631


In [15]:
len(gals[gals['StellarMass']>0.0]['StellarMass']*1.0E10/0.673)

44858

In [16]:
len(gals[gals['StellarMass']*1.0E10/0.673>10**8.5]['StellarMass']*1.0E10/0.673)

21122

# Cutting galaxies

In [17]:
gals2 =gals[gals['StellarMass']*1.0E10/0.673>10**8.5]

In [18]:
gals2['StellarMass']*1.0E10/0.673

array([  2.63575808e+09,   4.87133984e+08,   1.98012846e+10, ...,
         1.95747930e+09,   1.98673203e+09,   6.17012608e+08], dtype=float32)